In [7]:
#KIA_FAQ
# pip install requests beautifulsoup4
import requests
from bs4 import BeautifulSoup

url = "https://www.kia.com/content/cq:graphql/global/endpoint.json"

# questions
a_selector = ".cmp-accordion__title"
# user-agent
user_agent = 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/123.0.0.0 Safari/537.36'

def get_question_list():
    """
    FAQ questions 크롤링.

    aguments
    return
        DataFrame: 조회결과들을 담은 DataFrame(표)
    raise
        Exception: 처리 실패시 발생
    """
    # 1. 요청
    res = requests.post(url, headers={"user-agent":user_agent})
    params = {
        
        'query' : '''query($tag:String) {
            faqList (
                filter: {
                  tags: {
                    _expressions:  {
                      	value:$tag
                      	_operator:EQUALS
                    }
                  }
                  _path: {
                    _expressions: [
                        {
                            value: "/content/dam/kwp/kr/ko"
                           _operator: STARTS_WITH
                        }
                    ]
                    }
                }
        
            ) {
                items {
                    _path
                    question
                    answer {
                        html
                        plaintext
                    }
                    important
                    sortorder
                    created
                    assets {
                        ... on AssetModel {
                          title
                          asset {
                              __typename
                              ... on ImageRef {
                                _path
                              }
                          }
                          mobileAsset{
                              __typename
                              ... on ImageRef {
                                _path
                              }
                          }
                          imageBorder
                        }
                    }
                    tags
                }
            }
        }

    ''',
        'variables' : '{"tag": "kwp:kr/faq/top10"}'
    }
    res = requests.post(url, params=params)
    data = res.json()
    # 2. 응답 페이지에서 필요한 정보 추출
    if res.status_code == 200:
        return data
    else:
        raise Exception(f"요청 실패. 응답코드: {res.status_code}")      

data = get_question_list()
datata = data['data']['faqList']['items']
question_arr = []
for i in datata :
    question_arr.append([i['question'], i['answer']['html']])

import pandas as pd
faq = pd.DataFrame(question_arr, columns=["question" , "answer"])
faq.columns = ['Questions', 'Answer']

faq.to_csv('faq.csv',index=False)

# if __name__ == "__main__":
#     result = get_daum_news_list()
#     # from pprint import pprint
#     # pprint(result)
    
#     # 저장할 디렉토리를 생성
#     import os
#     from datetime import datetime
#     import pandas as pd
#     save_dir = "daum_news_list"
#     os.makedirs(save_dir, exist_ok=True)
    
#     # 저장할 파일명 - 특정 기간마다 크롤링 수행할 경우 실행 날짜/시간을 이용해서 만들어 준다.
#     d = datetime.now().strftime("%Y-%m-%d-%H-%M-%S")
#     file_path = f"{save_dir}/{d}.csv"
#     # DataFrame 생성ㅠㅠ
#     result_df = pd.DataFrame(result, columns=['제목', "링크주소"])
#     # csv 파일로 저장.
#     result_df.to_csv(file_path, index=False)

In [9]:
#db 생성
import pandas as pd
! pip install pandas sqlalchemy pymysql
from sqlalchemy import create_engine

# CSV 파일 불러오기
csv_file_path = 'faq.csv'  # CSV 파일 경로
df = pd.read_csv(csv_file_path)

# MySQL 데이터베이스 연결 설정
username = 'scott'  # MySQL 사용자 이름
password = 'tiger'  # MySQL 비밀번호
host = 'localhost'  # MySQL 서버 주소 (보통 localhost)
port = '3306'  # MySQL 포트 (기본값은 3306)
database = 'faq'  # 데이터베이스 이름

# MySQL 데이터베이스 연결 문자열 생성
connection_string = f'mysql+pymysql://{username}:{password}@{host}:{port}/{database}'
engine = create_engine(connection_string)

# DataFrame을 MySQL 데이터베이스에 저장
df.to_sql('faq_table', engine, if_exists='replace', index=False)

print("CSV 파일이 MySQL 데이터베이스에 저장되었습니다.")

CSV 파일이 MySQL 데이터베이스에 저장되었습니다.


In [43]:
#JEJU

import requests
from bs4 import BeautifulSoup

url = "https://jejuevservice.com/echarger"

# questions
# a_selector = "div > div.contentList > div"
a_selector = "div.title > div > span"

# user-agent
user_agent = 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/123.0.0.0 Safari/537.36'

#subPage > div > div > div.contentList > div > div.faq_title
#subPage > div > div > div.contentList > div

def get_faq_list():
   
    # 1. 요청
    res = requests.get(url, headers={"user-agent":user_agent})
    # 2. 응답 페이지에서 필요한 정보 추출
    if res.status_code == 200:
        soup = BeautifulSoup(res.text, "lxml")
        a_list = soup.select(a_selector)# [<a href="url">제목</a>, ...]
        result_list = []
        print(a_list)
        for a_tag in a_list:
            title = a_tag.get_text() 
            link = a_tag.get("href") 
            result_list.append([title.strip(), link])
        
        
        return result_list
    else:
        raise Exception(f"요청 실패. 응답코드: {res.status_code}")

result = get_faq_list()
print(result)

[<span class="table-cell">차량별 급속충전기 타입을 모르겠어요.</span>, <span class="table-cell">급속충전기 사용법이 궁금해요.</span>, <span class="table-cell">완속충전기 사용법이 궁금해요.</span>, <span class="table-cell">충전기 위치는 어떻게 확인하나요?</span>, <span class="table-cell">(SM3) 차량에서 커넥터가 안 빠져요.</span>, <span class="table-cell">충전기 커넥터 도어가 열리지 않아요.</span>, <span class="table-cell">충전기 화면에 비상정지라고 쓰여 있어요.</span>, <span class="table-cell">현재 충전하는 사람이 없는데 충전기화면에는 충전중이라고 나와요.</span>, <span class="table-cell">아이오닉인데 커넥터가 맞지 않아요</span>, <span class="table-cell">충전을 할 때마다 주행가능 거리가 다른데 차량에 문제가 있는건가요?</span>]
[['차량별 급속충전기 타입을 모르겠어요.', None], ['급속충전기 사용법이 궁금해요.', None], ['완속충전기 사용법이 궁금해요.', None], ['충전기 위치는 어떻게 확인하나요?', None], ['(SM3) 차량에서 커넥터가 안 빠져요.', None], ['충전기 커넥터 도어가 열리지 않아요.', None], ['충전기 화면에 비상정지라고 쓰여 있어요.', None], ['현재 충전하는 사람이 없는데 충전기화면에는 충전중이라고 나와요.', None], ['아이오닉인데 커넥터가 맞지 않아요', None], ['충전을 할 때마다 주행가능 거리가 다른데 차량에 문제가 있는건가요?', None]]


In [ ]:
from pprint import pprint
result = get_faq_list()
pprint(result)

# if __name__ == "__main__":
#     result = get_faq_list()
#     # from pprint import pprint
#     # pprint(result)
    
#     # 저장할 디렉토리를 생성
#     import os
#     from datetime import datetime
#     import pandas as pd
#     save_dir = "faq_list"
#     os.makedirs(save_dir, exist_ok=True)
    
#     # 저장할 파일명 - 특정 기간마다 크롤링 수행할 경우 실행 날짜/시간을 이용해서 만들어 준다.
#     d = datetime.now().strftime("%Y-%m-%d-%H-%M-%S")
#     file_path = f"{save_dir}/{d}.csv"
#     # DataFrame 생성
#     result_df = pd.DataFrame(result, columns=['제목', "링크주소"])
#     # csv 파일로 저장.
#     result_df.to_csv(file_path, index=False)

In [5]:
#GS

import requests
from bs4 import BeautifulSoup

url = "https://www.gscev.com/kr/community/faq/1?searchWord=&searchCate="

# questions
# a_selector = "div > div.contentList > div"
# a_selector = div.content-info > div > div.faq-content.ui-accordion > div > div.accordion-item
a_selector = "#faqForm > div.content-info > div > div.faq-content.ui-accordion > div > div > button > span > span"

# user-agent
user_agent = 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/123.0.0.0 Safari/537.36'

#subPage > div > div > div.contentList > div > div.faq_title
#subPage > div > div > div.contentList > div

def get_faq_list():
   
    # 1. 요청
    res = requests.get(url, headers={"user-agent":user_agent})
    # 2. 응답 페이지에서 필요한 정보 추출
    if res.status_code == 200:
        soup = BeautifulSoup(res.text, "lxml")
        a_list = soup.select(a_selector)# [<a href="url">제목</a>, ...]
        result_list = []
        print(a_list)
        for a_tag in a_list:
            title = a_tag.get_text() 
            link = a_tag.get("href") 
            result_list.append([title.strip(), link])
        
        
        return result_list
    else:
        raise Exception(f"요청 실패. 응답코드: {res.status_code}")

result = get_faq_list()
print(result)

#faqForm > div.content-info > div > div.faq-content.ui-accordion > div span

[<span class="tit">
<em>Q</em>충전 후 세차 / 세차 후 충전해도 안전한가요?
													</span>, <span class="tit">
<em>Q</em>눈/비오는 날도 충전 가능한가요?
													</span>, <span class="tit">
<em>Q</em>다른 차량이 충전중이던걸 제가 결제해버렸어요.어떻게 해야 할까요?
													</span>, <span class="tit">
<em>Q</em>다른 차량이 충전 중이었는데 모르고 충전을 중지시켜버렸어요.
													</span>, <span class="tit">
<em>Q</em>GS&amp;POINT 적립은 안되나요?
													</span>, <span class="tit">
<em>Q</em>충전기가 작동하지 않아요
													</span>, <span class="tit">
<em>Q</em>커넥터 도어가 열리지 않습니다.
													</span>, <span class="tit">
<em>Q</em>비상버튼이 눌려서 해제가 되지 않아요.
													</span>, <span class="tit">
<em>Q</em>충전기 선이 벗겨졌어요
													</span>, <span class="tit">
<em>Q</em>충전기에서 타는 냄새가 나요
													</span>]
[['Q충전 후 세차 / 세차 후 충전해도 안전한가요?', None], ['Q눈/비오는 날도 충전 가능한가요?', None], ['Q다른 차량이 충전중이던걸 제가 결제해버렸어요.어떻게 해야 할까요?', None], ['Q다른 차량이 충전 중이었는데 모르고 충전을 중지시켜버렸어요.', None], ['QGS&POINT 적립은 안되나요?', None], ['Q충전기가 작동하지 않아요', None], ['Q커넥터 도어가 열리지 않습니다.', None], ['Q비상버튼이 눌려서 